In [1]:
cd ../../apps/

e:\kaggle\pytorch-book\apps


In [2]:
import time
# from pathlib import Path

# from random import randint
# from matplotlib import pyplot as plt

import torch as np
from torchvision.utils import save_image

from models.CSA import CSA
from tools.toml import load_option

from tools.file import mkdir
from utils.torch_loader_all import Loader


def array2image(x):
    x *= 255
    x = x.detach().cpu().numpy()
    return x.astype('uint8').transpose((1, 2, 0))

def mask_op(mask):
    mask = mask.cuda()
    mask = mask[0][0]
    mask = np.unsqueeze(mask, 0)
    mask = np.unsqueeze(mask, 1)
    mask = mask.byte()
    return mask

## 模型定义

In [3]:
# 超参数设定
## 固定参数
epochs = 500
display_freq = 49
save_epoch_freq = 1

## 模型参数
alpha = 1
beta = 1


model_name = f'CSA-crop-{alpha}-{beta}'

In [4]:
base_opt = load_option('../options/base.toml')
model_opt = load_option('../options/train-new.toml')
model_opt.update(base_opt)
model_opt.update({'name': model_name}) # 设定模型名称
model = CSA(beta, **model_opt)

image_save_dir = model.save_dir / 'images'
mkdir(image_save_dir)

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyReLU(negat

In [5]:
opt = load_option('../origin/all.toml')
print(opt)
loader = Loader(**opt)

{'root': 'E:/kaggle/datasets/buildings', 'mask': 'D:/kaggle/dataset/mask/testing_mask_dataset', 'fine_size': 512, 'batch_size': 1}


In [6]:
# 训练阶段
start_epoch = 0
total_steps = 0
iter_start_time = time.time()
for epoch in range(start_epoch, epochs):
    epoch_start_time = time.time()
    epoch_iter = 0
    # 初始化数据集
    trainset = loader.trainset() # 训练集
    maskset = loader.maskset() # mask 数据集
    for image, mask in zip(trainset, maskset):
        mask = mask_op(mask)
        total_steps += model.batch_size
        epoch_iter += model.batch_size
        # it not only sets the input data with mask,
        #  but also sets the latent mask.
        model.set_input(image, mask)
        model.set_gt_latent()
        model.optimize_parameters()
        if total_steps % display_freq == 0:
            real_A, real_B, fake_B = model.get_current_visuals()
            # real_A=input, real_B=ground truth fake_b=output
            pic = (np.cat([real_A, real_B, fake_B], dim=0) + 1) / 2.0
            image_name = f"epoch{epoch}-{total_steps}-{alpha}.png"
            save_image(pic, image_save_dir/image_name, ncol=1)
        if total_steps % 100 == 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / model.batch_size
            print(
                f"Epoch/total_steps/alpha-beta: {epoch}/{total_steps}/{alpha}-{beta}", dict(errors))
    if epoch % save_epoch_freq == 0:
        print(f'保存模型 Epoch {epoch}, iters {total_steps} 在 {model.save_dir}')
        model.save(epoch)
    print(
        f'Epoch/Epochs {epoch}/{epochs-1} 花费时间：{time.time() - epoch_start_time}s')
    model.update_learning_rate()

Epoch/total_steps/alpha-beta: 0/100/1-1 {'G_GAN': 5.116807460784912, 'G_L1': 47.245216369628906, 'D': 1.578263759613037, 'F': 0.18597255647182465}
Epoch/total_steps/alpha-beta: 0/200/1-1 {'G_GAN': 4.580358505249023, 'G_L1': 79.05139923095703, 'D': 1.1153419017791748, 'F': 0.35055941343307495}
保存模型 Epoch 0, iters 283 在 ..\result\CSA-crop-1-1
Epoch/Epochs 0/499 花费时间：2079.9001491069794s
learning rate = 0.002
Epoch/total_steps/alpha-beta: 1/300/1-1 {'G_GAN': 5.272180557250977, 'G_L1': 86.1685791015625, 'D': 0.9750052690505981, 'F': 0.26144856214523315}
Epoch/total_steps/alpha-beta: 1/400/1-1 {'G_GAN': 3.490828275680542, 'G_L1': 77.38365173339844, 'D': 1.5139873027801514, 'F': 0.5771482586860657}
Epoch/total_steps/alpha-beta: 1/500/1-1 {'G_GAN': 4.772912502288818, 'G_L1': 43.340946197509766, 'D': 1.101444959640503, 'F': 0.22798380255699158}
保存模型 Epoch 1, iters 566 在 ..\result\CSA-crop-1-1
Epoch/Epochs 1/499 花费时间：2027.0861275196075s
learning rate = 0.002
Epoch/total_steps/alpha-beta: 2/600/1

KeyboardInterrupt: 